In [3]:
import pandas as pd
from sklearn.cluster import KMeans
from feature_engine.encoding import OneHotEncoder as fe_OneHotEncoder
from feature_engine.encoding import CountFrequencyEncoder
from sklearn.pipeline import Pipeline

In [4]:
df = pd.read_csv('salaries.csv')

In [5]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2022,SE,FT,Marketing Data Analyst,200000,USD,200000,GB,100,GB,S
1,2022,EN,FT,Data Scientist,96000,CAD,74378,CA,100,CA,L
2,2022,SE,FT,Data Science Lead,165000,USD,165000,US,50,US,S
3,2022,EN,FT,Data Scientist,27000,GBP,33599,GB,50,GB,L
4,2022,SE,FT,Data Engineer,185900,USD,185900,US,0,US,M


In [6]:
df.describe()

,work_year,salary,salary_in_usd,remote_ratio
count,1195.000000,1.195000e+03,1195.000000,1195.000000
mean,2021.684519,2.495210e+05,122041.142259,66.736402
std,0.584604,1.136720e+06,66487.800833,44.035085
min,2020.000000,2.324000e+03,2324.000000,0.000000
25%,2021.000000,8.000000e+04,73961.000000,0.000000
50%,2022.000000,1.300000e+05,120000.000000,100.000000
75%,2022.000000,1.755500e+05,160237.500000,100.000000
max,2022.000000,3.040000e+07,600000.000000,100.000000


In [7]:
encodingPipeline = Pipeline([  # Define the pipeline
    ('oneHotEnc', fe_OneHotEncoder(top_categories=None, variables=["experience_level", "employment_type", "salary_currency", "company_size"], drop_last=True)),
    ('countEncoder', CountFrequencyEncoder(encoding_method='count', variables=["job_title", "employee_residence", "company_location"]))
])

encodingPipeline.fit(df)  # Fit the pipeline
df = encodingPipeline.transform(df)  # Save imputed data to original X_train

In [8]:
df.head()

,work_year,job_title,salary,salary_in_usd,employee_residence,remote_ratio,company_location,experience_level_SE,experience_level_EN,experience_level_MI,...,salary_currency_SGD,salary_currency_CNY,salary_currency_PLN,salary_currency_CHF,salary_currency_JPY,salary_currency_HUF,salary_currency_MXN,salary_currency_TRY,company_size_S,company_size_L
0,2022,2,200000,200000,78,100,82,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,2022,310,96000,74378,36,100,37,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,2022,1,165000,165000,795,50,823,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2022,310,27000,33599,78,50,82,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,2022,276,185900,185900,795,0,823,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
k_means = KMeans(n_clusters=3)
k_means.fit(df)

labels = k_means.predict(df)

c:\Users\leand\anaconda3\envs\cds106\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=5.
  warnings.warn(
